<a href="https://colab.research.google.com/github/TheGEN1U5/DLventures/blob/main/nlp/seq2seq/vanillaWMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.32.4
    Uninstalling huggingface-hub-0.32.4:
      Successfully uninstalled huggingface-hub-0.32.4
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3

In [1]:
import datasets

In [2]:
dataset = datasets.load_dataset("wmt/wmt14", "de-en", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/280M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/273M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/474k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [3]:
from tokenizers import Tokenizer

In [4]:
import pandas as pd
df = pd.DataFrame(dataset["translation"])

### train

In [ ]:
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, Punctuation, Sequence as Pre_Seq
from tokenizers.normalizers import Lowercase, Sequence

tkz = Tokenizer(BPE(unk_token="[UNK]"))
tkz.normalizer = Lowercase()
tkz.pre_tokenizer = Pre_Seq([Whitespace(), Punctuation()])
trainer = BpeTrainer(vocab_size=15000, special_tokens=["[UNK]", "[PAD]", "[EOS]", "[SOS]", "[MASK]"])

In [ ]:
training_data = df["de"].to_list() + df["en"].to_list()
tkz.train_from_iterator(training_data, trainer)
import numpy as np
np.random.shuffle(training_data)

In [ ]:
tkz.train_from_iterator(training_data, trainer)

### load

In [5]:
import requests
text = requests.get("https://raw.githubusercontent.com/TheGEN1U5/DLventures/refs/heads/main/nlp/seq2seq/tokenizer.json").text
with open("tokenizer.json", "w") as f:
    f.write(text)
tkz = Tokenizer.from_file("tokenizer.json")

In [6]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [7]:
tkz.encode("I love you!").ids

[51, 7546, 3757, 5]

In [51]:
df = df[:100000]

df["de"] = df["de"].apply(lambda x: torch.tensor(tkz.encode(x).ids, dtype=torch.long))
df["en"] = df["en"].apply(lambda x: torch.tensor(tkz.encode(x).ids, dtype=torch.long))


TypeError: TextInputSequence must be str

In [52]:
df["de_len"] = df["de"].apply(lambda x: torch.tensor(x.shape[0], dtype=torch.long))
df["en_len"] = df["en"].apply(lambda x: torch.tensor(x.shape[0], dtype=torch.long))

In [17]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    en, de, en_len, de_len = zip(*batch)
    en_len = torch.stack(en_len)
    de_len = torch.stack(de_len)
    padded_en = pad_sequence(en, batch_first=True, padding_value=1)
    padded_de = pad_sequence(de, batch_first=True, padding_value=1)
    de = torch.cat(3 * torch.ones((de.shape[0], 1)), de, 4 * torch.ones((de.shape[0], 1)))
    return padded_en, padded_de, en_len, de_len

In [18]:
train_loader = DataLoader(list(zip(df["en"].to_numpy(), df["de"].to_numpy(), df["en_len"].to_numpy(), df["de_len"].to_numpy())), batch_size=500, shuffle=True, collate_fn=collate_fn)

In [ ]:
class SeqTranslator(nn.Module):
    def __init__(self, embed_dim, hidden_dim, vocab_size, num_layers):
        super(SeqTranslator, self).__init__()
        self.embed_layer = nn.Embedding(vocab_size, embed_dim)
        self.encoder = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.decoder = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, y, x_len, y_len):
        x_emb = self.embed_layer(x)
        x_pack = pack_padded_sequence(x_emb, x_len, batch_first=True, enforce_sorted=False)
        y_emb = self.embed_layer(y)
        y_pack = pack_padded_sequence(y_emb, y_len, batch_first=True, enforce_sorted=False)
        o, (hn, cn) = self.encoder(x_pack)
        d, _ = self.decoder(y_pack, (hn, cn))
        logits = self.output_layer(d)

    def translate(self, x):
        self.eval()

        x_emb = self.embed_layer(x)
        o, (hn, cn) = self.encoder(x_emb)
        tokens = 3 * torch.ones((x.shape[0], 1), device=x.device)
        max_len = 100
        for _ in range(max_len):
            y_emb = self.embed_layer(tokens[-1])
            d, (hn, cn) = self.decoder(y_emb, (hn, cn))
            logits = self.output_layer(d)
            probs = nn.functional.softmax(logits, dim=-1)
            token = torch.argmax(probs, dim=-1).unsqueeze(0)
            if token == 4:
                break
            tokens = torch.cat(tokens, token)
        return tokens


In [22]:
tokens = [1,2,3]

emb = nn.Embedding(10, 4)
a = torch.tensor(3)
emb(torch.tensor(tokens[-1]))

tensor([ 0.6696, -1.8580, -0.0564, -0.4507], grad_fn=<EmbeddingBackward0>)

In [29]:
tk = torch.tensor(tokens, dtype=torch.long)

In [49]:
reversed(tk)

tensor([3, 2, 1])

In [41]:
torch.cat((tk, torch.argmax(tk).unsqueeze(0)))

tensor([1, 2, 3, 2])

In [50]:
3 * torch.ones((5, 1))

tensor([[3.],
        [3.],
        [3.],
        [3.],
        [3.]])